**1) Приведите еще примеры метрик для оценки рекомендаций/ранжирования (можно взять из интернета, или ваши знания)**

1) **F-score** (F1, F1_k)- взвешенное гармоническое среднее precision и recall

2) **MAR_k** - средний recall по всем юзерам (доля рекомендованных товаров среди купленных)

3) **MAE** - оценка точности рекомендаций, разница между реальной оценкой пользователя и предсказанной оценкой

4) **RSME** - оценка точности рекомендаций, корень из квадрата разницы между реальной оценкой и предсказанной. Сильнее штрафует большие ошибки, чем MAE

5) **AVERAGE RECIPROCAL HIT RANK** - вариация hit-rate, учитывает ранк выбранных рекомендаций)

6) **Coverage** - процент возможных рекомендаций, которые система может предсказать, и насколько быстро новые товары появляются в рекомендациях

7) **Diversity** - насколько широк ассортимент рекомендаций, выдаваемых системой

8) **Churn** - насколько часто меняются рекомендации (например, после негативной оценки к-л товара пользователем)

**2) Доделать все функции, где стоит комментарий "сделать дома"**

In [1]:
import numpy as np

In [2]:
def hit_rate_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    flags = np.isin(bought_list, recommended_list)
    return (flags.sum() > 0) * 1

In [3]:
recommended_list = [156, 1134, 27, 1543, 533, 32, 11, 43, 3345] #id товаров
bought_list = [521, 32, 143, 991]
hit_rate_at_k(recommended_list, bought_list)

0

In [4]:
def precision_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(recommended_list)


In [5]:
recommended_list = [143,156, 1134, 991, 27, 1543, 3345, 533, 11, 43] #id товаров
bought_list = [521, 32, 143, 991]
precision_at_k(recommended_list, bought_list, k=3)

0.3333333333333333

In [11]:
def recall_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(bought_list)


def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    prices_bought = np.array(prices_bought)
    recommend_list = np.array(recommended_list)[:k]
    prices_recommended = np.array(prices_recommended)[:k]
    
    flags = np.isin(recommend_list, bought_list)
    recall = np.dot(flags, prices_recommended).sum() / prices_bought.sum()
    
    return recall

In [12]:
recommended_list = [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43]
bought_list = [521, 32, 143, 991]
prices_recommended = [5, 2, 4, 11, 8, 10, 15, 7, 3, 9]
prices_bought = [6, 8, 5, 11]
recall_at_k(recommended_list, bought_list, k=5)

0.5

In [13]:
money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5)

0.5333333333333333

In [14]:
def map_k(recommended_list, bought_list, k=5):
    num_users = len(bought_list)
    sum_ap_k = 0
    for i in range(num_users):
        b_list = np.array(bought_list[i])
        r_list = np.array(recommended_list[i])[:k]
        relevant_indexes = np.nonzero(np.isin(r_list, b_list))[0]
        if len(relevant_indexes) == 0:
            continue
    
        amount_relevant = len(relevant_indexes)
        sum_ = sum([precision_at_k(r_list, b_list, k=index_relevant+1) for index_relevant in relevant_indexes])
        sum_ap_k += sum_/amount_relevant
    
    return sum_ap_k / num_users

In [15]:
recommended_list_3_users = [[143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43], 
                    [1134, 533, 14, 4, 15, 1543, 1, 99, 27, 3345],
                    [991, 3345, 27, 533, 43, 143, 1543, 156, 1134, 11]
                    ]

bought_list_3_users = [[521, 32, 143],  # юзер 1
                       [143, 156, 991, 43, 11], # юзер 2
                       [1,2]] # юзер 3
map_k(recommended_list_3_users, bought_list_3_users, k=5)

0.3333333333333333

In [16]:
def ndcg_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    flags = np.isin(recommended_list, bought_list)
    discount_i = np.append([1, 2], [np.log2(i) for i in range(3, k+1)])
    dcg = 1/k * sum(flags / discount_i)
    ideal_dcg = 1/k * sum(np.ones(len(flags)) / discount_i)
    return dcg / ideal_dcg    

In [17]:
recommended_list = [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43]
bought_list = [521, 32, 143, 991]
ndcg_at_k(recommended_list, bought_list, k=5)

0.489938890671454

In [18]:
def reciprocal_rank(recommended_list, bought_list, k=1):
    num_users = len(bought_list)
    k_u = []
    for i in range(num_users):
        b_list = np.array(bought_list[i])
        r_list = np.array(recommended_list[i])[:k]
        if sum(np.isin(r_list, b_list)) ==0:
            k_u.append(0)
            continue
        k_u.append(1 / (np.nonzero(np.isin(r_list, b_list))[0][0] +1))
    if k_u ==[]:
        return 0
    return np.array(k_u).mean()        

In [19]:
reciprocal_rank(recommended_list_3_users, bought_list_3_users, k=5)

0.3333333333333333